# Sensitivity analysis on previous work

Training data from all actions, test subjects and aspect angles, excluding the data from person A, which
is used for testing, and excluding the walking samples.

In [2]:
# Needed to allow editing using PyCharm
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import os
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report,confusion_matrix
import csv
from keras.models import load_model
from src.models.original_models import mixed_angle_A

Using TensorFlow backend.


In [4]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

In [5]:
# Check if using CoLab or local pc and set file path accordingly
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    
else:
    FILE_PATH = "C:/Users/macka/Google Drive/Level-4-Project/data/"

In [6]:
def load_batch(fpath, label_key='labels'):
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            # added check as otherwise tries to decode a string
            if type(k) is not str:
                del(d[k])
                d[k.decode("utf8")] = v
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 1, 75, 75)
    return data, labels

In [7]:
def load_data(data_folder):
    path = FILE_PATH + data_folder

    nb_train_samples = 43450

    X_train = np.zeros((nb_train_samples, 1, 75, 75), dtype="uint8")
    y_train = np.zeros((nb_train_samples,), dtype="uint8")

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        data, labels = load_batch(fpath)
        X_train[(i-1)*8690:i*8690, :, :, :] = data
        y_train[(i-1)*8690:i*8690] = labels

    fpath = os.path.join(path, 'test_batch')
    X_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (X_train, y_train), (X_test, y_test)

In [11]:
batch_size = 100
nb_classes = 7
nb_epoch = 100
nb_epoch = 1  # originally 100
data_augmentation = False

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB


# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = load_data("mixed_angle_A")
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = mixed_angle_A.make_model(img_rows, img_cols, nb_classes)

X_train shape: (43450, 1, 75, 75)
43450 train samples
8690 test samples


In [12]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    hist = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)

    y_pred = model.predict_classes(X_test)
    print(y_pred)
    target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#     sys.stdout = open('/home/aleksandar/new_sets/mixed_angle_A/results/report_mixed_angle_A_64_128_.txt', "w")
    print("CLASSIFICATION REPORT:")
    print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
    print ("\n")
    print("CONFUSION MATRIX:")
    print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

#     with open('/home/aleksandar/new_sets/mixed_angle_A/results/graphs_mixed_angle_A_64_128.csv', "w") as f:
#        w = csv.writer(f)
#        train_loss = hist.history['loss']
#        val_loss = hist.history['val_loss']
#        train_acc = hist.history['acc']
#        val_acc = hist.history['val_acc']
#        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#        w.writerow("train_loss, train_acc, val_loss, val_acc")
#        for val in helplist:
#           w.writerow([val])

	
#     model.save('/home/aleksandar/new_sets/mixed_angle_A/results/model_mixed_angle_A_64_128.h5')

else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    hist = model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

    y_pred = model.predict_classes(X_test)
    print(y_pred)
    target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
#     sys.stdout = open('/home/aleksandar/sets/cifar_noninitialised_A/results/report_noninitialised_A_2_4.txt', "w")
    print("CLASSIFICATION REPORT:")
    print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
    print("\n")
    print("CONFUSION MATRIX:")
    print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

#     with open('/home/aleksandar/sets/cifar_noninitialised_A/results/graphs_noninitialised_A_2_4.csv', "w") as f:
#        w = csv.writer(f)
#        train_loss = hist.history['loss']
#        val_loss = hist.history['val_loss']
#        train_acc = hist.history['acc']
#        val_acc = hist.history['val_acc']
#        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
#        w.writerow("train_loss, train_acc, val_loss, val_acc")
#        for val in helplist:
#           w.writerow([val])
#     model.save('/home/aleksandar/sets/cifar_noninitialised_A/results/model_noninitialised_A_2_4.h5')

Not using data augmentation.


Train on 43450 samples, validate on 8690 samples
Epoch 1/1


  100/43450 [..............................] - ETA: 17:15 - loss: 1.9504 - acc: 0.1000

  200/43450 [..............................] - ETA: 13:12 - loss: 1.9493 - acc: 0.1250

  300/43450 [..............................] - ETA: 11:51 - loss: 1.9485 - acc: 0.1200

...

18300/43450 [===========>..................] - ETA: 5:28 - loss: 1.1001 - acc: 0.5548

18400/43450 [===========>..................] - ETA: 5:27 - loss: 1.0957 - acc: 0.5565

18500/43450 [===========>..................] - ETA: 5:26 - loss: 1.0914 - acc: 0.5584

18600/43450 [===========>..................] - ETA: 5:24 - loss: 1.0877 - acc: 0.5598

18700/43450 [===========>..................] - ETA: 5:23 - loss: 1.0835 - acc: 0.5616